In [4]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import numpy as np
import pandas as pd
import math
import glob

from model import LSTMDriver
from model import HYPERPARAMS

import logging
logging.basicConfig(filename='training.log',level=logging.DEBUG)

In [5]:
PROJECT_ROOT = '../../../..'
LEARNED_DRIVER = 'snakeoil_miner/data'
DIFFICULTY = 'easy'


TRAINING_FILES = glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, DIFFICULTY, '*.csv']))
TRAINING_DATA = {}
for FILE in TRAINING_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    TRAINING_DATA[FILE] = DF.values

VALIDATION_FILES = glob.glob('/'.join([PROJECT_ROOT, LEARNED_DRIVER, 'validation', '*.csv']))
VALIDATION_DATA = {}
for FILE in VALIDATION_FILES:
    DF = pd.read_csv(FILE, index_col=False)
    VALIDATION_DATA[FILE] = DF.values
    
CUDA = torch.cuda.is_available()
if CUDA:
    DTYPE = torch.cuda.FloatTensor
else:
    DTYPE = torch.FloatTensor

In [6]:
def save_checkpoint(state, is_best, filepath='latest_checkpoint.pth.tar'):
    torch.save(state, 'checkpoints/' + filepath)
    if is_best:
        torch.save(state, 'checkpoints/best_checkpoint.pth.tar')

In [7]:
def train(training_data, model, criterion, optimzier):
    loss = 0
    model.train(mode=True)
    for key in training_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        optimizer.zero_grad()
        track_sequence = training_data[key]

        targets = track_sequence[:, 0:3]
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets)).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs),  requires_grad=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        track_loss.backward()
        optimizer.step()

        loss += track_loss.data[0]
    return loss

In [8]:
def validate(validation_data, model, criterion):
    loss = 0
    model.train(mode=False)
    for key in validation_data:
        logging.info('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        print('--- Parsing track {}-{}'.format(key.split('/')[-2], key.split('/')[-1]))
        
        model.hidden_state = model.init_hidden()
        track_sequence = validation_data[key]

        targets = track_sequence[:, 0:3]
        inputs = track_sequence[:, 3:]

        targets_variable = autograd.Variable(torch.Tensor(targets), volatile=True).type(DTYPE)
        inputs_variable = autograd.Variable(torch.Tensor(inputs), volatile=True).type(DTYPE)

        outputs_variable = model(inputs_variable)

        track_loss = criterion(outputs_variable, targets_variable)

        loss += track_loss.data[0]
    return loss

In [ ]:
if CUDA:
    model = LSTMDriver(HYPERPARAMS.INPUT_SIZE,
                       HYPERPARAMS.LSTM_HIDDEN_SIZE,
                       HYPERPARAMS.HIDDEN_LAYER_SIZE,
                       HYPERPARAMS.DROPOUT_PROB,
                       HYPERPARAMS.NUM_LAYERS,
                       HYPERPARAMS.TARGET_SIZE,
                       HYPERPARAMS.BATCH_SIZE).cuda()
    criterion = nn.MSELoss().cuda()
else:
    model = LSTMDriver(HYPERPARAMS.INPUT_SIZE,
                       HYPERPARAMS.LSTM_HIDDEN_SIZE,
                       HYPERPARAMS.HIDDEN_LAYER_SIZE,
                       HYPERPARAMS.DROPOUT_PROB,
                       HYPERPARAMS.NUM_LAYERS,
                       HYPERPARAMS.TARGET_SIZE,
                       HYPERPARAMS.BATCH_SIZE)
    criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=HYPERPARAMS.LEARNING_RATE)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', verbose=True)

min_loss = math.inf
losses = {
  'training': [],
  'validation': []
}

for epoch in np.arange(HYPERPARAMS.NUM_EPOCHS):
    logging.info('Epoch [%d/%d]' %(epoch+1, HYPERPARAMS.NUM_EPOCHS))
    print('Epoch [%d/%d]' %(epoch+1, HYPERPARAMS.NUM_EPOCHS))
    
    is_best = False

    training_loss = train(TRAINING_DATA, model, criterion, optimizer)
    logging.info('--- TRAINING LOSS: %f' % training_loss)
    print('--- TRAINING LOSS: %f' % training_loss)

    validation_loss = validate(VALIDATION_DATA, model, criterion)
    logging.info('--- VALIDATION LOSS: %f' % validation_loss)
    print('--- VALIDATION LOSS: %f' % validation_loss)

    if validation_loss < min_loss:
        logging.info('--- --- best model found so far: %f' % validation_loss)
        print('--- --- best model found so far: %f' % validation_loss)
        min_loss = validation_loss
        is_best = True

    losses['training'].append(training_loss)
    losses['validation'].append(validation_loss)

    save_checkpoint({
          'epoch': epoch + 1,
          'state_dict': model.state_dict(),
          'min_loss': min_loss,
          'optimizer' : optimizer.state_dict(),
      }, is_best)

    scheduler.step(validation_loss)
    logging.info('-------------------------------------------------------')
    print('-------------------------------------------------------')

Epoch [1/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 1.095590
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.370581
--- --- best model found so far: 0.370581
-------------------------------------------------------
Epoch [2/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.401893
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.307543
--- --- best model found so far: 0.307543
-------------------------------------------------------
Epoch [3/150]
--- Parsing track easy-race_03.csv
---

--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.177117
-------------------------------------------------------
Epoch [19/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.180067
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.168609
-------------------------------------------------------
Epoch [20/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.171346
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDAT

--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.125971
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.140078
-------------------------------------------------------
Epoch [38/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.125187
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.149461
-------------------------------------------------------
Epoch [39/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.123675
-

--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.107528
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.138236
-------------------------------------------------------
Epoch [57/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.106832
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.138211
Epoch    56: reducing learning rate of group 0 to 1.0000e-05.
-------------------------------------------------------
Epoch [58/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--

--- VALIDATION LOSS: 0.138653
-------------------------------------------------------
Epoch [75/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.105336
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.138656
-------------------------------------------------------
Epoch [76/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.105379
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.138658
-------------------------------------------------------
Epoch [77/150]
--- Parsing track easy-race_03.cs

--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.138666
-------------------------------------------------------
Epoch [94/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.105445
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDATION LOSS: 0.138666
-------------------------------------------------------
Epoch [95/150]
--- Parsing track easy-race_03.csv
--- Parsing track easy-race_00.csv
--- Parsing track easy-race_01.csv
--- Parsing track easy-race_02.csv
--- TRAINING LOSS: 0.105520
--- Parsing track validation-race_30.csv
--- Parsing track validation-race_33.csv
--- Parsing track validation-race_32.csv
--- Parsing track validation-race_31.csv
--- VALIDAT